In [1]:
# ============================================================
# 02 - Feature Engineering
# Fonte: SUSEP AUTOSEG (2019-2021)
# Autor: Arthur Pontes Motta
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurações visuais
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 5)

# Caminhos
PROCESSED_PATH = '../data/processed'
FIGURES_PATH = '../reports/figures'

# ============================================================
# 1. CARREGAR DADOS PROCESSADOS
# ============================================================

df = pd.read_parquet(os.path.join(PROCESSED_PATH, 'autoseg_limpo.parquet'))

print(f"Shape: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
df.head()

Shape: (12244283, 27)
Colunas: ['regiao', 'cod_modelo', 'ano_modelo', 'sexo', 'faixa_etaria', 'exposicao', 'premio', 'is_media', 'freq_roubo', 'indeniz_roubo', 'freq_colisao_parcial', 'indeniz_colisao_parcial', 'freq_colisao_total', 'indeniz_colisao_total', 'freq_incendio', 'indeniz_incendio', 'freq_outras', 'indeniz_outras', 'ano', 'semestre', 'sexo_desc', 'faixa_etaria_desc', 'freq_colisao', 'indeniz_colisao', 'sinistro_total', 'indeniz_total', 'periodo']


,regiao,cod_modelo,ano_modelo,sexo,faixa_etaria,exposicao,premio,is_media,freq_roubo,indeniz_roubo,...,indeniz_outras,ano,semestre,sexo_desc,faixa_etaria_desc,freq_colisao,indeniz_colisao,sinistro_total,indeniz_total,periodo
0,12,025185-2,2016,M,3,6.747945,8112.761084,59889.521135,0,0.0,...,345.0,2019,2,Masculino,Entre 36 e 45 anos,0,0.0,3,345.0,2019-S2
1,12,025214-0,2015,J,3,1.076712,1232.988800,32457.320949,0,0.0,...,0.0,2019,2,Jurídico,Entre 36 e 45 anos,0,0.0,0,0.0,2019-S2
2,12,001388-9,2014,M,4,0.495890,884.724060,46033.998554,0,0.0,...,0.0,2019,2,Masculino,Entre 46 e 55 anos,0,0.0,0,0.0,2019-S2
3,07,001453-2,2019,J,5,5.030137,15386.125233,130695.640343,0,0.0,...,0.0,2019,2,Jurídico,Maior que 55 anos,0,0.0,0,0.0,2019-S2
4,21,033080-9,2012,N,0,0.495890,1291.726807,134337.992224,0,0.0,...,0.0,2019,2,Não informado,Não informada,0,0.0,0,0.0,2019-S2


In [2]:
# ============================================================
# 2. FEATURE ENGINEERING
# ============================================================

# --- Filtros para modelagem ---
# Focar em pessoa física (excluir Jurídico e Não informado)
# e apenas registros com faixa etária informada
df_model = df[
    (df['sexo'].isin(['M', 'F'])) &
    (df['faixa_etaria'] > 0)
].copy()

print(f"Shape após filtros: {df_model.shape}")
print(f"Removidos: {len(df) - len(df_model):,} registros")

# --- Variável resposta: frequência relativa de colisão ---
# sinistros / exposição (taxa de sinistralidade)
df_model['freq_colisao_rel'] = df_model['freq_colisao'] / df_model['exposicao']

# --- Variável resposta: severidade média de colisão ---
# indenização / número de sinistros (custo médio por sinistro)
# apenas onde houve sinistro
df_model['severidade_colisao'] = np.where(
    df_model['freq_colisao'] > 0,
    df_model['indeniz_colisao'] / df_model['freq_colisao'],
    np.nan
)

# --- Idade do veículo ---
ANO_REF = 2021
df_model['ano_modelo_int'] = pd.to_numeric(df_model['ano_modelo'], errors='coerce')
df_model['idade_veiculo'] = ANO_REF - df_model['ano_modelo_int']
df_model['idade_veiculo'] = df_model['idade_veiculo'].clip(0, 30)

# --- IS relativa (proxy de valor do veículo) ---
df_model['log_is_media'] = np.log1p(df_model['is_media'])

print("\nEstatísticas das variáveis resposta:")
print(df_model[['freq_colisao_rel', 'severidade_colisao']].describe().round(2))

Shape após filtros: (9106060, 27)
Removidos: 3,138,223 registros

Estatísticas das variáveis resposta:
       freq_colisao_rel  severidade_colisao
count        9106060.00           923816.00
mean               0.09            10887.00
std                3.13            18110.62
min                0.00                1.00
25%                0.00             2569.62
50%                0.00             5417.00
75%                0.00            11488.50
max             1825.00          1198502.00


In [3]:
# ============================================================
# 3. TRATAMENTO DE OUTLIERS E ENCODING
# ============================================================

# Remover outliers extremos de frequência
# (grupos com exposição muito baixa distorcem o modelo)
p99_freq = df_model['freq_colisao_rel'].quantile(0.99)
p99_sev  = df_model['severidade_colisao'].quantile(0.99)

print(f"Limite freq (p99): {p99_freq:.4f}")
print(f"Limite severidade (p99): {p99_sev:.2f}")

df_model = df_model[df_model['freq_colisao_rel'] <= p99_freq].copy()
print(f"\nShape após remoção de outliers: {df_model.shape}")

# --- Encoding de variáveis categóricas ---

# Sexo: binário
df_model['sexo_bin'] = (df_model['sexo'] == 'M').astype(int)

# Faixa etária: ordinal (já está como 1-5)
# 1=18-25, 2=26-35, 3=36-45, 4=46-55, 5=55+

# Região: one-hot encoding
df_model['regiao'] = df_model['regiao'].astype(str).str.strip()
regiao_dummies = pd.get_dummies(df_model['regiao'], prefix='regiao', drop_first=True)
df_model = pd.concat([df_model, regiao_dummies], axis=1)

print("\nRegiões disponíveis:", sorted(df['regiao'].unique()))
print("Dummies criadas:", [c for c in df_model.columns if c.startswith('regiao_')])

# --- Dataset final para modelagem ---
features_freq = (
    ['sexo_bin', 'faixa_etaria', 'idade_veiculo', 'log_is_media'] +
    [c for c in df_model.columns if c.startswith('regiao_')]
)

print(f"\nFeatures do modelo: {features_freq}")
print(f"Shape final: {df_model.shape}")

Limite freq (p99): 1.7633
Limite severidade (p99): 86195.05

Shape após remoção de outliers: (9015020, 32)

Regiões disponíveis: ['  ', ' .', '0', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '5', '6', '7', '8', '9', '99']
Dummies criadas: ['regiao_.', 'regiao_0', 'regiao_00', 'regiao_01', 'regiao_02', 'regiao_03', 'regiao_04', 'regiao_05', 'regiao_06', 'regiao_07', 'regiao_08', 'regiao_09', 'regiao_1', 'regiao_10', 'regiao_11', 'regiao_12', 'regiao_13', 'regiao_14', 'regiao_15', 'regiao_16', 'regiao_17', 'regiao_18', 'regiao_19', 'regiao_2', 'regiao_20', 'regiao_21', 'regiao_22', 'regiao_23', 'regiao_24', 'regiao_25', 'regiao_26', 'regiao_27', 'regiao_28', 'regiao_29', 'regiao_3', 'regiao_30', 'regiao_31', 'regiao_32', 'regiao_33', 'regiao_34', 'regiao_35', 'reg

In [4]:
# ============================================================
# 4. DECODIFICAÇÃO E LIMPEZA DE REGIÕES
# ============================================================

# Carregar tabela auxiliar de regiões
reg_path = os.path.join('../data/raw', 'Autoseg2021A', 'auto_reg.csv')
df_reg = pd.read_csv(reg_path, sep=';', encoding='latin-1')
df_reg.columns = df_reg.columns.str.strip()
df_reg['codigo'] = df_reg[df_reg.columns[0]].astype(str).str.strip().str.zfill(2)
df_reg['regiao_desc'] = df_reg[df_reg.columns[1]].str.strip()

print("Regiões válidas:")
print(df_reg[['codigo', 'regiao_desc']])

# Padronizar região no df_model
df_model['regiao_cod'] = df_model['regiao'].astype(str).str.strip().str.zfill(2)

# Manter apenas regiões válidas
regioes_validas = df_reg['codigo'].tolist()
df_model = df_model[df_model['regiao_cod'].isin(regioes_validas)].copy()
print(f"\nShape após filtro de regiões válidas: {df_model.shape}")
print(f"Regiões no modelo: {sorted(df_model['regiao_cod'].unique())}")

# Refazer dummies com regiões limpas
regiao_dummies = pd.get_dummies(df_model['regiao_cod'], prefix='regiao', drop_first=True)

# Remover dummies antigas e adicionar novas
cols_remover = [c for c in df_model.columns if c.startswith('regiao_')]
df_model = df_model.drop(columns=cols_remover)
df_model = pd.concat([df_model, regiao_dummies], axis=1)

# Atualizar lista de features
features_freq = (
    ['sexo_bin', 'faixa_etaria', 'idade_veiculo', 'log_is_media'] +
    [c for c in df_model.columns if c.startswith('regiao_')]
)

print(f"\nFeatures finais ({len(features_freq)}): {features_freq}")
print(f"Shape final: {df_model.shape}")

Regiões válidas:
   codigo                                      regiao_desc
0      01           RS - Met. Porto Alegre e Caxias do Sul
1      02                              RS - Demais regiões
2      03                    SC - Met. Florianópolis e Sul
3      04                                       SC - Oeste
4      05                   SC - Blumenau e demais regiões
5      06         PR - F.Iguaþu-Medianeira-Cascavel-Toledo
6      07                               PR - Met. Curitiba
7      08                              PR - Demais regiões
8      09                   SP - Vale do Paraíba e Ribeira
9      10            SP - Litoral Norte e Baixada Santista
10     11                           SP - Met. de São Paulo
11     12                             SP - Grande Campinas
12     13    SP - Ribeirão Preto e Demais Mun. de Campinas
13     14                           MG - Triângulo mineiro
14     15                                         MG - Sul
15     16  MG - Met.BH-Centro Oeste-Zon

In [6]:
# ============================================================
# 5. SALVAR DATASET DE MODELAGEM
# ============================================================

# Salvar dataset de modelagem
output = os.path.join('../data/processed', 'autoseg_model.parquet')
df_model.to_parquet(output, index=False)
print(f"✓ Dataset de modelagem salvo: {output}")
print(f"  Shape: {df_model.shape}")
print(f"  Features: {len(features_freq)}")
print(f"\nDistribuição da variável resposta (freq_colisao_rel):")
print(df_model['freq_colisao_rel'].describe().round(4))

✓ Dataset de modelagem salvo: ../data/processed\autoseg_model.parquet
  Shape: (8868406, 73)
  Features: 44

Distribuição da variável resposta (freq_colisao_rel):
count    8.868406e+06
mean     3.090000e-02
std      1.415000e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.763300e+00
Name: freq_colisao_rel, dtype: float64
